In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]
plt.gray()
import os
import cv2
import h5py
from metavision_core.event_io import EventsIterator, RawReader
from metavision_sdk_core import BaseFrameGenerationAlgorithm, ColorPalette
import torch
import torch.nn as nn
import torch.nn.functional as F
import snntorch as snn
from snntorch import surrogate
import pandas as pd
from network import SCNN
from simulator import *

<Figure size 800x600 with 0 Axes>

# Generate the Data

## Generate the Velocities

In [ ]:
W = 14.0      # field width (m)
L = 11.0       # field length (m)

field = Field(W, L, 2.4, 1)


Vmax = 5    # maximum speed (m/s)
R = 0.11       # radius of the ball (m) (11 - 11.5 cm)
margin = 2
constraints_go_in = [
    lambda params: 0 < params["lx"] < params["field"].W,  # Landing X within bounds
    lambda params: params["ly"] < params["field"].L,  # Landing Y within bounds
    lambda params: params["tf"] > 0.3,          # Minimum flight time
    lambda params: 0< params["t_y0"] < 5,          # Maximum time to reach X
    lambda params: - (params["field"].GW/2) < params["x_y0"] - params["field"].center[0] < params["field"].GW/2,  # Ball should go through the goal or near it
    lambda params: params["R"] <= params["z_y0"] < params["field"].GH/2,  # Ball should go through the goal or near it
]

constraints_almost_go_in = [
    lambda params: 0 < params["lx"] < params["field"].W,  # Landing X within bounds
    lambda params: params["ly"] < params["field"].L,  # Landing Y within bounds
    lambda params: params["tf"] > 0.3,          # Minimum flight time
    lambda params: 0< params["t_y0"] < 5,          # Maximum time to reach X
    lambda params: - (params["field"].GW/2 + margin) < params["x_y0"] - params["field"].center[0] < - params["field"].GW/2 or params["field"].GW/2 < params["x_y0"] - params["field"].center[0] < params["field"].GW/2 + margin or params["field"].GH/2 <= params["z_y0"] < params["field"].GH/2 + margin,  # Ball should go through the goal or near it
]

constraints_go_out = [
    lambda params: 0 < params["lx"] < params["field"].W,  # Landing X within bounds
    lambda params: params["ly"] < params["field"].L,  # Landing Y within bounds
    lambda params: params["tf"] > 0.3,          # Minimum flight time
    lambda params: 0< params["t_y0"] < 5,          # Maximum time to reach X
]

traj_go_in_gen = TrajectoryGenerator(constraints_go_in, R, field, Vmax)
traj_almost_go_in_gen = TrajectoryGenerator(constraints_almost_go_in, R, field, Vmax)
traj_go_out_gen = TrajectoryGenerator(constraints_go_out, R, field, Vmax)

fps = 200

camera_pos = np.array([field.center[0], 0, 0.7])          # Camera position in world space
camera_target = np.array([field.center[0], field.center[1], camera_pos[2]])        # Where the camera is looking (center of field) It would be good to transform this into an angle and assume verticality
orientation = Camera.get_orientation(camera_pos, camera_target)
img_width, img_height = 640, 480               # Image resolution (in pixels)
focal_length = 800                             # Focal length in pixels (adjust as needed)

camera = Camera(camera_pos, orientation, focal_length, img_width, img_height, fps)

In [ ]:
init_go_in = np.array(traj_go_in_gen.generate_velocities())
init_almost_go_in = np.array(traj_almost_go_in_gen.generate_velocities())
init_go_out = np.array(traj_go_out_gen.generate_velocities())

## Generate the Videos in IsaacSIM

In [ ]:
# Different Lightings
# Robot moving in the background
# Goalie moving

# Convert Videos to Events

# Train SNN
## Load events

In [3]:
import glob
dir_paths = glob.glob('Event_camera_data\\*.raw', recursive=True)
dir_paths

['Event_camera_data\\recording_2025-02-20_13-38-07.raw',
 'Event_camera_data\\recording_2025-02-20_13-41-38.raw',
 'Event_camera_data\\recording_2025-02-20_13-45-16.raw',
 'Event_camera_data\\recording_2025-02-20_13-47-09.raw']

In [4]:
path_to_events= 'Event_camera_data\\recording_2025-02-20_13-38-07.raw'
raw_data = RawReader(path_to_events)
height, width = raw_data.get_size()

In [5]:
def read_time_volume(raw_data, start_t_us, delta_t_us):
    raw_data.reset()
    if start_t_us > raw_data.current_time:
        raw_data.seek_time(start_t_us)
    events_vol = raw_data.load_delta_t(delta_t_us)
    # events_vol['t'] -= int(start_t_us)
    return events_vol
def viz_events(events, height, width):
    img = np.zeros((height, width, 3), dtype=np.uint8)
    # check that all events are within width-heigh
    ids = np.logical_and(events['y'] < height, events['x'] < width) 
    events = events[ids]
    events_plus = events[events['p']>0]
    events_minus = events[events['p']==0]
    img[events_plus['y'], events_plus['x'], 0] = 255
    img[events_minus['y'], events_minus['x'], 2] = 255
    return img
def gen_imgs(raw_data, start_t_us, end_t_us, delta_t_us):
    imgs = []

    for t in range(int(start_t_us), int(end_t_us), int(delta_t_us)):
        events = read_time_volume(raw_data, t, delta_t_us)
        img = viz_events(events, height, width)
        imgs.append(img)
    imgs = torch.tensor(imgs, dtype=torch.float32)
    return imgs

In [6]:
imgs = gen_imgs(raw_data, 56e6, 58e6, 10e3)
training_ratio = 0.8
training_size = int(training_ratio*imgs.shape[0])
train_data = imgs[:training_size]
test_data = imgs[training_size:]
print(train_data.shape, test_data.shape)

torch.Size([160, 720, 1280, 3]) torch.Size([40, 720, 1280, 3])


C:\Users\realenriquem\AppData\Local\Temp\ipykernel_2744\2324584307.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\bld\libtorch_1738203676394\work\torch\csrc\utils\tensor_new.cpp:281.)
  imgs = torch.tensor(imgs, dtype=torch.float32)


## Initialize the SNN

In [ ]:
n_goal_divisions = 10
scnn = SCNN(n_goal_divisions)
scnn.train(train_data, test_data, n_epochs=10)

# Evaluate SNN